In [1]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
import urllib
import requests
import json
import numpy as np
from datetime import datetime
import os


# 키즈 카페 정보 크롤링

In [ ]:
# 페이지 수 (1부터 시작)
pages = 1

# 데이터 저장 리스트
kids_cafe_list = []

# 페이지 반복
while pages < 20:
    # URL 생성 (페이지 번호 변경)
    KIDS_URL = f'https://icare.seoul.go.kr/icare/user/kidsCafe/BD_selectKidsCafeList.do?q_hiddenVal=1&q_fcltyId=&q_rowPerPage=5&q_currPage={pages}&q_sortName=&q_sortOrder=&q_searchVal=&q_useAge='

    # 페이지 가져오기
    response = requests.get(KIDS_URL)
    if response.status_code != 200:
        break  # 페이지 로드 실패 시 종료

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(response.content, 'html.parser')

    # 키즈카페 목록 가져오기
    elements = soup.select("div.board_kidscafe > div.kidscafe_wrap")

    # 페이지가 끝났으면 종료
    if not elements:
        break

    for elem in elements:
        data = {
            "키즈카페 이름": None,
            "자치구": None,
            "주소": None,
            "이용연령_min": None,
            "이용연령_max": None,
            "이용정원_개인": None,
            "이용정원_단체": None,
            "예약 신청 URL": None
        }

        # 키즈카페 이름
        try:
            name = elem.find('h5').text.strip()
            data["키즈카페 이름"] = name
        except:
            pass

        # 이용정원 (개인, 단체)
        try:
            capacity_text = elem.select_one("dd").text.strip()
            # 줄바꿈 문자로 분리하고 공백을 제거
            capacity_lines = [line.strip() for line in capacity_text.split("\n") if line.strip()]
            for i in range(len(capacity_lines)):
                if "개인" in capacity_lines[i]:
                    data["이용정원_개인"] = capacity_lines[i + 1].replace("명", "").strip()
                if "단체" in capacity_lines[i]:
                    data["이용정원_단체"] = capacity_lines[i + 1].replace("명", "").strip()

        except:
            pass

        # 이용연령_min
        try:
            age = elem.select_one("dt.age + dd").text.strip()
            data["이용연령_min"] = re.findall(r'\d+', age)[0]
            data["이용연령_max"] = re.findall(r'\d+', age)[1]
        except:
            pass

        # 주소
        try:
            address = elem.select_one("dt.age + dd + dt + dd").text.strip()
            data["주소"] = address
            data['자치구'] = address.split(' ')[1]
        except:
            pass

        # 예약 신청 URL
        try:
            reservation_link = elem.select_one("a.lg_btn")["href"]
            if "icare" in reservation_link:
                reservation_link = "https://icare.seoul.go.kr" + reservation_link

            data["예약 신청 URL"] = reservation_link
        except:
            pass

        # 리스트에 추가
        kids_cafe_list.append(data)

    # 페이지 번호 증가
    pages += 1


In [ ]:
# 데이터프레임 변환
df = pd.DataFrame(kids_cafe_list)

# 행 전체가 None(NaN) 값이면 삭제
df = df.dropna(how="all")

# CSV 파일로 저장
df.to_csv("kids_cafe_list_all.csv", index=False, encoding='cp949')

print("데이터 저장 완료: kids_cafe_list_all.csv")

데이터 저장 완료: kids_cafe_list_all.csv


In [ ]:
# 'icare' 포함데이터만 필터링
df_filtered = df[df["예약 신청 URL"].str.contains("icare", na=False)]

# CSV 파일로 저장
df_filtered.to_csv("kids_cafe_list.csv", index=False, encoding='utf-8-sig')

print("필터링된 데이터 저장 완료: kids_cafe_list.csv", len(df_filtered))

필터링된 데이터 저장 완료: kids_cafe_list.csv 75


In [ ]:
# 'icare'가 포함되지 않은 데이터만 필터링 --> 크롤링 불가능 키즈 카페
df_filtered = df[~df["예약 신청 URL"].str.contains("icare", na=False)]

# CSV 파일로 저장
df_filtered.to_csv("kids_cafe_list_other.csv", index=False, encoding='utf-8-sig')

print("필터링된 데이터 저장 완료: kids_cafe_list_other.csv", len(df_filtered))

필터링된 데이터 저장 완료: kids_cafe_list_other.csv 3


# 예약 정보 크롤링

In [ ]:
# ---------------설정 변수-------------------------------
# 날짜 설정 ex) month = '03', want_day = [29] --> 컬럼네임 0329_1회

# 월 설정 (문자열)
month = '04'

# 예약 현황 크롤링할 날짜 설정 (숫자, 리스트)
want_day = [5]
# ------------------------------------------------------

crawling_dt = datetime.now().strftime('%Y-%m-%d-%H')

# 키즈카페 데이터 저장 리스트
kids_cafe_data = []

df = pd.read_csv('kids_cafe_list.csv')

# URL 리스트
kids_cafe_urls = df['예약 신청 URL'].loc[:]

for url in kids_cafe_urls:

    # URL이 없으면 건너뛰기
    if not url:
        continue

    # 페이지 요청
    response = requests.get(url)
    
    # 응답 상태 코드가 200(성공)이 아닌 경우 건너뛰기
    if response.status_code != 200:
        print(f"URL 로드 실패: {url}")
        continue
    
    soup = BeautifulSoup(response.content, 'html.parser')

    # 키즈카페 이름 추출
    name = soup.select_one("#container > div > div.write_ty01.m_write_ty01.wr_type > table > tbody > tr:nth-child(1) > td:nth-child(2)").text.strip()

    # 키즈카페 예약 현황 추출
    elements = soup.select("#calendar > tbody > tr")

    # 키즈카페 데이터 저장용 딕셔너리
    data = {"키즈카페 이름": name, "예약 신청 URL": url}

    for element in elements:
        title_td = element.find_all('td')
        for title in title_td:
            try:
                # 날짜 추출
                date_number = int(re.findall(r'\d+', title.get('title'))[0])
                
                # 원하는 날짜인지 확인
                if date_number in want_day:
                    for p in title.find_all('p'):
                        session_info = p.get_text(strip=True)
                        session_info = ' '.join(session_info.split())  # 불필요한 공백 제거
                        
                        # "1회프로그램" → 프로그램
                        if "프로그램" in session_info:
                            match = re.match(r'(\d+회)프로그램', session_info)
                            if match:
                                session = match.group(1)
                                column_name = f"{month}{date_number}_{session}"  # 예: '29_1회'
                                data[column_name] = "프로그램"

                        # "1회개인0" 또는 "1회단체1" 처리
                        else:
                            match = re.match(r'(\d+회)(개인|단체|공용)(\d+)', session_info)
                            if match:
                                session, user_type, count = match.groups()
                                column_name = f"{month}{date_number}_{session}"  # 예: '0329_3회'
                                data[column_name] = f"{user_type}_{count}"  # 예: '개인_0' or '단체_1'

            except:
                continue

    # 데이터 리스트에 추가
    kids_cafe_data.append(data)

# 데이터프레임 생성
df = pd.DataFrame(kids_cafe_data)

df['crawling_dt'] = crawling_dt

# CSV 파일 저장
df.to_csv(f"{"_".join(map(str, want_day))}_re_{crawling_dt}.csv", index=False, encoding='cp949')

print(f"데이터 저장 완료: {"_".join(map(str, want_day))}_re_{crawling_dt}.csv")

데이터 저장 완료: 1_2_re_2025-04-01-13.csv


In [ ]:
df

,name,예약 신청 URL,0329_1회,0329_2회,0329_3회,0329_4회,0329_5회,0330_1회,0330_2회,0330_3회,0330_4회,0330_5회,crawling_dt
0,서울형 키즈카페 시립 1호점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,2025-03-28-20
1,서울형 키즈카페 시립 뚝섬자벌레점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-28-20
2,서울형 키즈카페 시립 목동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,개인_0,개인_0,NaN,NaN,NaN,NaN,NaN,2025-03-28-20
3,서울형 키즈카페 강남구 역삼1동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_1,개인_0,NaN,NaN,개인_0,개인_0,개인_0,NaN,NaN,2025-03-28-20
4,서울형 키즈카페 강동구 고덕2동점(아이·맘 강동),https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,NaN,NaN,개인_0,개인_0,개인_0,NaN,NaN,2025-03-28-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,은평아이맘놀이터 수색동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-28-20
71,서울형 키즈카페 종로구 혜화동점(종로 혜명 아이들 상상놀이터),https://icare.seoul.go.kr/icare/user/kidsCafeR...,NaN,프로그램,프로그램,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-28-20
72,서울형 키즈카페 중구 중림동점(노리몽땅),https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_17,개인_25,개인_30,개인_33,NaN,개인_33,개인_26,개인_34,개인_33,NaN,2025-03-28-20
73,서울형 키즈카페 중랑구 망우본동점(중랑실내놀이터 양원),https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,개인_58,NaN,개인_14,개인_26,개인_19,개인_61,NaN,2025-03-28-20
